In [ ]:
import pandas as pd
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("viber1/indian-law-dataset")

# Convert to Pandas DataFrame
df = pd.DataFrame(dataset["train"])

# Print available columns
print(df.columns)



Index(['Instruction', 'Response'], dtype='object')


In [ ]:
# Select the correct column (adjust based on actual dataset)
df = df[['Instruction']]  # Change 'text' if needed based on print(df.columns) output


In [ ]:
print(df.head())  # View first few rows


                                         Instruction
0  What is the difference between a petition and ...
1     When should a writ petition be filed in India?
2  What is the procedure for filing a plaint in a...
3  What are the common reliefs sought through a p...
4  Can a plaint be amended after it has been file...


In [ ]:
import pandas as pd
from datasets import load_dataset

# Load dataset from Hugging Face
dataset = load_dataset("viber1/indian-law-dataset")

# Convert dataset to Pandas DataFrame
df = pd.DataFrame(dataset["train"])

# Print available columns
print(df.columns)

# Display sample data
print(df.head())


Index(['Instruction', 'Response'], dtype='object')
                                         Instruction  \
0  What is the difference between a petition and ...   
1     When should a writ petition be filed in India?   
2  What is the procedure for filing a plaint in a...   
3  What are the common reliefs sought through a p...   
4  Can a plaint be amended after it has been file...   

                                            Response  
0  A petition is a formal request submitted to a ...  
1  A writ petition in India should be filed when ...  
2  To file a plaint in a civil case in Indiayou m...  
3  Public interest litigation (PIL) petitions in ...  
4  Yesa plaint can be amended in a civil case in ...  


In [ ]:
df = df[['Response']]  # Keep only the response column (legal case texts)

# Drop empty values
df = df.dropna()

print(df.head())  # View processed data


                                            Response
0  A petition is a formal request submitted to a ...
1  A writ petition in India should be filed when ...
2  To file a plaint in a civil case in Indiayou m...
3  Public interest litigation (PIL) petitions in ...
4  Yesa plaint can be amended in a civil case in ...


In [ ]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained embedding model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Convert legal texts into embeddings
embeddings = model.encode(df["Response"].tolist(), convert_to_tensor=True)

print("Embedding shape:", embeddings.shape)  # Should be (num_cases, 384)


Embedding shape: torch.Size([24607, 384])


In [12]:
import faiss
import numpy as np

# Convert embeddings to NumPy
embeddings_np = embeddings.cpu().numpy()

# Create FAISS index
index = faiss.IndexFlatL2(embeddings_np.shape[1])

# Add embeddings to FAISS index
index.add(embeddings_np)

print("FAISS index size:", index.ntotal)  # Should match number of cases


FAISS index size: 24607


In [14]:
def search_legal_query(query, top_k=3):
    # Convert query to embedding
    query_embedding = model.encode([query], convert_to_tensor=True).cpu().numpy()

    # Search in FAISS
    distances, indices = index.search(query_embedding, top_k)

    # Get top K matching legal texts
    results = df.iloc[indices[0]]["Response"].tolist()

    return results

# Example user query
query = "What is the punishment for theft under IPC?"
retrieved_texts = search_legal_query(query)

# Display retrieved results
for i, text in enumerate(retrieved_texts):
    print(f"🔹 Result {i+1}: {text[:500]}...\n")  # Print first 500 characters


🔹 Result 1: Section 379 specifies the punishment for theft. A person convicted of theft may face imprisonment of either description for a term that can extend up to three years, or a fine, or both....

🔹 Result 2: Section 382 deals with theft committed after making preparations for causing death, hurt, restraint, or fear of such consequences. The punishment for this offence is rigorous imprisonment for a term that may extend to ten years, along with a fine. The section provides illustrations to clarify scenarios where this offence might apply....

🔹 Result 3: Section 356 pertains to assault or criminal force in an attempt to commit theft of property carried by a person. The punishment may include imprisonment for up to two years, a fine, or both. It aims to deter offenses related to attempting theft from a person....

